In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers -q
!pip install seqeval -q
! pip install datasets -q

In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json

with open('drive/MyDrive/NLP/data.json') as f:
  data = json.load(f)
  
X_train, X_val, X_test, y_train, y_val, y_test = data['X_train'], data['X_val'], data['X_test'], data['y_train'], data['y_val'], data['y_test'] 

In [ ]:
from datasets import load_dataset

In [ ]:
train_dataset = load_dataset('json', data_files='drive/MyDrive/NLP/train_new.json', field='data', streaming=True)['train']
val_dataset = load_dataset('json', data_files='drive/MyDrive/NLP/val_new.json', field='data')['train']
# test_dataset = load_dataset('json', data_files='drive/MyDrive/NLP/test.json', field='data')['train']

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
print(val_dataset[:10])
print(tokenizer(val_dataset[0]['text']))
# print(len(val_dataset))

{'text': ['It was her credit card_comma_ I borrowed it without telling her', ' It was storming the other night and the power went out.  ', 'Future Nobel prize winning macro economists of /r/politics explain that high gas prices can be solved by nationalizing oil companies ... and because company executives are all mini-Hitlers Or, you know, you could simply allow the current oil companies to drill in more places', '# The only thing that matters\n\nHere\'s something you need to understand first. In most rural districts, pretty much any idiot can get hired to a position and stay there as long as they don\'t piss off parents too much. The people hiring you don\'t really understand what you teach, and the parents don\'t understand or care what quality teaching is, but they care if you say something that offends their backward sensibilities. What that effectively means is that your ability to teach and stay on has more to do with being in sync with the community, who are usually racist and 

In [ ]:
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = "max_length", truncation=True, max_length=32, return_tensors='pt')

In [ ]:
train_dataset = train_dataset.map(tokenization, batched='True', batch_size=16)
val_dataset = val_dataset.map(tokenization, batched='True', batch_size=16)
# test_dataset = test_dataset.map(tokenization, batched='True', batch_size=16)

In [ ]:
import torch
from torch.utils.data import IterableDataset, DataLoader

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
class StreamDataset(IterableDataset):
  def __init__(self, text, labels) -> None:
     self.item = [{'text': tokenization(x).to(device), 'label':torch.tensor(y, device=device)} for x,y in zip(text,labels)]

  def __iter__(self):
    return iter(self.item)

In [ ]:
# train_dataset = StreamDataset(X_train, y_train)
# val_dataset = StreamDataset(X_val, y_val)
# test_dataset = StreamDataset(X_test, y_test)

# assert isinstance(train_dataset, torch.utils.data.IterableDataset)

train_dataset=train_dataset.with_format('torch')
# val_dataset=val_dataset.with_format('torch')
# test_dataset=test_dataset.with_format('torch')

In [ ]:
from huggingface_hub import HfApi

# repo = Repository(local_dir="drive/MyDrive/NLP/topic_classifier")
api = HfApi()
api.create_repo(repo_id="sentientconch/intent_classifier_short_sent",
    repo_type="model",
    private=True
)

RepoUrl('https://huggingface.co/sentientconch/intent_classifier_short_sent', endpoint='https://huggingface.co', repo_type='model', repo_id='sentientconch/intent_classifier_short_sent')

In [ ]:
batch_size = 16
epochs = 1

training_args = TrainingArguments(output_dir="intent_classifier_short_sent",
    max_steps=32*epochs/batch_size,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size/2,
    evaluation_strategy="steps",
    logging_strategy = 'steps',
    save_strategy='steps',
    logging_steps=1,
    save_steps=1,
    eval_steps=1,
    disable_tqdm=False,
    push_to_hub=True
)

In [ ]:
batch_size = 128
epochs = 3

training_args = TrainingArguments(output_dir="intent_classifier_short_sent",
    max_steps=308273*epochs/batch_size,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="steps",
    logging_strategy = 'steps',
    save_strategy='steps',
    logging_steps=2409,
    save_steps=2409,
    eval_steps=2409,
    disable_tqdm=False,
    push_to_hub=True
)

In [ ]:
model = model.to(device)

In [ ]:
label_names=list(range(3))
label_names

[0, 1, 2]

In [ ]:
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p

    #select predicted index with maximum logit for each token
    # predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label)]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label)]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

<ipython-input-16-c65a6e0480e3>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
def preprocess_logits_for_metrics(logits, labels):
  pred_ids = torch.argmax(logits, dim=-1)
  return pred_ids, labels

In [ ]:
!pip install evaluate -q
import evaluate

In [ ]:
import numpy as np
acc = evaluate.load("accuracy")
prec = evaluate.load("precision")
rec = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy':acc.compute(predictions=predictions, references=labels), 'precision':prec.compute(predictions=predictions, references=labels, average='weighted'), 'recall':rec.compute(predictions=predictions,references=labels,average='weighted')}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


/content/intent_classifier_short_sent is already a clone of https://huggingface.co/sentientconch/intent_classifier_short_sent. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall
2409,0.223400,0.170189,{'accuracy': 0.934},{'precision': 0.9368420749048213},{'recall': 0.934}
4818,0.131200,0.177244,{'accuracy': 0.9333333333333333},{'precision': 0.9355077441895966},{'recall': 0.9333333333333333}


Trainer is attempting to log a value of "{'accuracy': 0.934}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9368420749048213}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.934}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9333333333333333}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9355077441895966}" of type <class 'dict'> for k

TrainOutput(global_step=7226, training_loss=0.1466743074359572, metrics={'train_runtime': 5263.0652, 'train_samples_per_second': 175.719, 'train_steps_per_second': 1.373, 'total_flos': 1.520746220702016e+16, 'train_loss': 0.1466743074359572, 'epoch': 2.33})

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "{'accuracy': 0.9406666666666667}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.941740168939677}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.9406666666666667}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.18705610930919647,
 'eval_accuracy': {'accuracy': 0.9406666666666667},
 'eval_precision': {'precision': 0.941740168939677},
 'eval_recall': {'recall': 0.9406666666666667},
 'eval_runtime': 5.3942,
 'eval_samples_per_second': 556.149,
 'eval_steps_per_second': 4.449,
 'epoch': 2.33}